In [2]:
read_save_path = 'D:/Record/長庚LAMP+TFO/成果/文本/tt/' # tt __TFO__

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer

chrome_options=Options()
chrome_options.add_argument('--headless')#無介面操作
driver = webdriver.Chrome(chrome_options=chrome_options)



PMCID_list = []
with open(read_save_path+'TFO_Target_ID.txt','r') as PMCID_file :
    for line in PMCID_file :
        PMCID_list.append(line.replace('\n',''))

PMCID_Title = []
PMCID_author = []
PMCID_year = []
PMCID_PMCID = []
PMCID_PMID = []
PMCID_journal = []
PMCID_raw_index = []
PMCID_index = []
PMCID_NE = []
PMCID_NE_Gene = []
PMCID_NE_Disease = []
PMCID_NE_Chemical = []
PMCID_NE_Mutation = []
PMCID_NE_Species = []
PMCID_NE_Cellline = []

Success_ID = []
Success_PMID = []
Success_PMCID = []
Fail_ID = []

print('--target PMCID : '+str(len(PMCID_list)))

running_number = 1

for each_PMCID_list in PMCID_list :

    driver.get('https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocxml?pmcids='+each_PMCID_list+'&concepts=')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        raw_string = ''
        string = ''
        NE_string = ''
        
        Gene_NE_string = ''
        Disease_NE_string = ''
        Chemical_NE_string = ''
        Mutation_NE_string = ''
        Species_NE_string = ''
        Cellline_NE_string = ''
                
        passages = soup.find_all('passage')
        for passage in passages :
            raw_string += '<<'+passage.find('infon', {'key': re.compile("section_type")}).text+'>>\n'
            string += '<<'+passage.find('infon', {'key': re.compile("section_type")}).text+'>>\n'
            try:
                raw_string += (passage.find('text').text+'\n\n')
                string += (passage.find('text').text+'\n\n')
            except:
                raw_string += ('\n\n')
                string += ('\n\n')
            annotations = passage.find_all('annotation')
            for annotation in annotations :
                NE = annotation.find('text')
                Type = annotation.find('infon', {'key': re.compile("type")})
                string += ('NE--'+NE.text+' '+Type.text+'\n')
                NE_string += NE.text+'\n'
                if Type.text == 'Gene' :
                    Gene_NE_string += NE.text+'\n'
                elif Type.text == 'Disease' :
                    Disease_NE_string += NE.text+'\n'
                elif Type.text == 'Chemical' :
                    Chemical_NE_string += NE.text+'\n'
                elif Type.text == 'Mutation' :
                    Mutation_NE_string += NE.text+'\n'
                elif Type.text == 'Species' :
                    Species_NE_string += NE.text+'\n'
                elif Type.text == 'Cellline' :
                    Cellline_NE_string += NE.text+'\n'
                
            if annotations == [] :
                string += '\n'
            else:
                string += '\n\n'
            raw_string += '\n'
        
        Title = soup.find('text')
        author = soup.find('infon', {'key': re.compile("authors")})
        year = soup.find('infon', {'key': re.compile("year")})
        ID = soup.find('id')
        other_id = soup.find('infon', {'key': re.compile("article-id_pmid")}) 
        journal = soup.find('infon', {'key': re.compile("journal")}) 

        Title_text = Title.text
        author_text = author.text
        year_text = year.text
        journal_text = journal.text
                
        PMCID_index.append(string)
        PMCID_raw_index.append(raw_string)
        PMCID_NE.append(NE_string)
        PMCID_Title.append(Title_text)
        PMCID_author.append(author_text)
        PMCID_year.append(year_text)
        PMCID_journal.append(journal_text)
        
        PMCID_NE_Gene.append(Gene_NE_string)
        PMCID_NE_Disease.append(Disease_NE_string)
        PMCID_NE_Chemical.append(Chemical_NE_string)
        PMCID_NE_Mutation.append(Mutation_NE_string)
        PMCID_NE_Species.append(Species_NE_string)
        PMCID_NE_Cellline.append(Cellline_NE_string)
                
                
#         if other_id!= None :
#             PMCID_PMCID.append('PMC'+ID.text)
#             PMCID_PMID.append('PMID'+other_id.text)
#         else:
#             other_id = soup.find('infon', {'key': re.compile("article-id_pmc")}) 
#             PMCID_PMCID.append(other_id.text)
#             PMCID_PMID.append('PMID'+ID.text)
        
        Success_ID.append(each_PMCID_list)
        Success_PMCID.append(each_PMCID_list)            
    except:
        driver.get("https://pubmed.ncbi.nlm.nih.gov/")
        driver.find_element_by_id("id_term").click()
        driver.find_element_by_id("id_term").clear()
        driver.find_element_by_id("id_term").send_keys(each_PMCID_list)
        driver.find_element_by_xpath("//form[@id='search-form']/div/div/div/button/span").click()
        try:
            each_PMID_list = driver.find_element_by_xpath('//*[@id="full-view-identifiers"]/li[1]/span/strong').text
            driver.get('https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocxml?pmids='+each_PMID_list+'&concepts=')
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            try:
                raw_string = ''
                string = ''
                NE_string = ''
                
                Gene_NE_string = ''
                Disease_NE_string = ''
                Chemical_NE_string = ''
                Mutation_NE_string = ''
                Species_NE_string = ''
                Cellline_NE_string = ''

                passages = soup.find_all('passage')
                for passage in passages :
                    raw_string += '<<'+passage.find('infon', {'key': re.compile("type")}).text+'>>\n'
                    string += '<<'+passage.find('infon', {'key': re.compile("type")}).text+'>>\n'
                    try:
                        raw_string += (passage.find('text').text+'\n\n')
                        string += (passage.find('text').text+'\n\n')
                    except:
                        raw_string += ('\n\n')
                        string += ('\n\n')
                    annotations = passage.find_all('annotation')
                    for annotation in annotations :
                        NE = annotation.find('text')
                        Type = annotation.find('infon', {'key': re.compile("type")})
                        string += ('NE--'+NE.text+'\t'+Type.text+'\n')
                        NE_string += NE.text+'\n'
                        
                        if Type.text == 'Gene' :
                            Gene_NE_string += NE.text+'\n'
                        elif Type.text == 'Disease' :
                            Disease_NE_string += NE.text+'\n'
                        elif Type.text == 'Chemical' :
                            Chemical_NE_string += NE.text+'\n'
                        elif Type.text == 'Mutation' :
                            Mutation_NE_string += NE.text+'\n'
                        elif Type.text == 'Species' :
                            Species_NE_string += NE.text+'\n'
                        elif Type.text == 'Cellline' :
                            Cellline_NE_string += NE.text+'\n'
                        
                    if annotations == [] :
                        string += '\n'
                    else:
                        string += '\n\n'
                    raw_string += '\n'

                Title = soup.find('text')
                author = soup.find('infon', {'key': re.compile("authors")})
                year = soup.find('infon', {'key': re.compile("year")})
                ID = soup.find('id')
                other_id = soup.find('infon', {'key': re.compile("article-id_pmid")}) 
                journal = soup.find('infon', {'key': re.compile("journal")}) 
                
                
                Title_text = Title.text
                author_text = author.text
                year_text = year.text
                journal_text = journal.text

                PMCID_index.append(string)
                PMCID_raw_index.append(raw_string)
                PMCID_NE.append(NE_string)
                PMCID_Title.append(Title_text)
                PMCID_author.append(author_text)
                PMCID_year.append(year_text)
                PMCID_journal.append(journal_text)
                
                PMCID_NE_Gene.append(Gene_NE_string)
                PMCID_NE_Disease.append(Disease_NE_string)
                PMCID_NE_Chemical.append(Chemical_NE_string)
                PMCID_NE_Mutation.append(Mutation_NE_string)
                PMCID_NE_Species.append(Species_NE_string)
                PMCID_NE_Cellline.append(Cellline_NE_string)
                
#                 if other_id!= None :
#                     PMCID_PMCID.append('PMC'+ID.text)
#                     PMCID_PMID.append('PMID'+other_id.text)

#                 else:
#                     other_id = soup.find('infon', {'key': re.compile("article-id_pmc")}) 
#                     PMCID_PMCID.append(other_id.text)
#                     PMCID_PMID.append('PMID'+ID.text)
                
                Success_ID.append(each_PMCID_list)
                Success_PMID.append(each_PMCID_list)
            except:
                Fail_ID.append(each_PMCID_list)
        except:
            Fail_ID.append(each_PMCID_list)
    print('('+str(running_number)+')',end = ' ')
    running_number += 1
    
# count #
print('\n')
print('Success ID --'+str(len(Success_ID)))
print('Success_PMCID --'+str(len(Success_PMCID)))
print('Success_PMID --'+str(len(Success_PMID)))
print('Fail ID --'+str(len(Fail_ID)))

driver.quit()



C:\Users\Tingchun.TC.Hung\Anaconda3\envs\NLP\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


WebDriverException: Message: Service chromedriver unexpectedly exited. Status code was: 3221225794


In [28]:
Data = pd.DataFrame([Success_ID,PMCID_Title,PMCID_year,PMCID_journal,PMCID_author,PMCID_index,PMCID_raw_index,PMCID_NE,PMCID_NE_Gene,PMCID_NE_Disease,PMCID_NE_Chemical,PMCID_NE_Mutation,PMCID_NE_Species,PMCID_NE_Cellline],index=['PMID','Title','Year','Journal','Author','Title & Abstract & NE','Title & Abstract','NE','PMCID_NE_Gene','PMCID_NE_Disease','PMCID_NE_Chemical','PMCID_NE_Mutation','PMCID_NE_Species','PMCID_NE_Cellline']).T # PMCID_author, 'Author',

In [29]:
Data

,PMID,Title,Year,Journal,Author,Title & Abstract & NE,Title & Abstract,NE,PMCID_NE_Gene,PMCID_NE_Disease,PMCID_NE_Chemical,PMCID_NE_Mutation,PMCID_NE_Species,PMCID_NE_Cellline
0,PMC3532560,Local delivery of gene-modifying triplex-formi...,2013,J. Invest. Dermatol.; 2013 Mar; 133(3) 685-691...,"Rogers FA, Hu RH, Milstone LM",<<TITLE>>\nLocal delivery of gene-modifying tr...,<<TITLE>>\nLocal delivery of gene-modifying tr...,CD1\nkeratin 5\nHairless\nhairless\ntransgenic...,CD1\nkeratin 5\nHairless\nhairless\nHPRT\nbeta...,back skin\nback skin\nback epidermis\ntoxicity...,TFO\nPNA-\nTFO\nAntp\ntriplex-forming oligonuc...,,transgenic mice\ntransgenic mice\nmice\nmice\n...,
1,PMC3188747,Alternative for anti-TNF antibodies for arthri...,2011,Mol. Ther.; 2011 Oct; 19(10) 1887-95. doi:10.1...,"Paquet J, Henrionnet C, Pinzano A, Vincourt JB...",<<title>>\nAlternative for anti-TNF antibodies...,<<title>>\nAlternative for anti-TNF antibodies...,TNF\narthritis\nTumor necrosis factor-alpha\nT...,TNF\nTumor necrosis factor-alpha\nTNF-alpha\nT...,arthritis\ninflammatory diseases\narthritis\nr...,triplex-forming oligonucleotide\nTFO\nTFO\nTFO...,,patients\nrat\n,
2,PMC373298,Triplex-forming oligonucleotide target sequenc...,2004,Nucleic Acids Res.; 2004; 32(1) 354-60,"Goñi JR, de la Cruz X, Orozco M",<<title>>\nTriplex-forming oligonucleotide tar...,<<title>>\nTriplex-forming oligonucleotide tar...,Triplex-forming oligonucleotide\nhuman\nhuman\...,,,Triplex-forming oligonucleotide\ntriplex-formi...,,human\nhuman\n,
3,PMC309654,In vivo transcription of a progesterone-respon...,1993,Nucleic Acids Res.; 1993 Jun 25; 21(12) 2789-96,"Ing NH, Beekman JM, Kessler DJ, Murphy M, Jaya...",<<title>>\nIn vivo transcription of a progeste...,<<title>>\nIn vivo transcription of a progeste...,triplex-forming oligonucleotide\nOligonucleoti...,progesterone receptor\n,,triplex-forming oligonucleotide\nOligonucleoti...,,,
4,PMC407813,Enhanced cellular uptake of a triplex-forming ...,2004,Nucleic Acids Res.; 2004; 32(7) 2102-12,"Santhakumaran LM, Thomas T, Thomas TJ",<<title>>\nEnhanced cellular uptake of a tripl...,<<title>>\nEnhanced cellular uptake of a tripl...,triplex-forming oligonucleotide\npolypropyleni...,c-myc\n,"breast, prostate and ovarian cancer\nbreast ca...",triplex-forming oligonucleotide\npolypropyleni...,,,
5,PMC2536773,Sequence-specific triple helix formation with ...,2007,Biochemistry; 2007 Oct 09; 46(40) 11240-52,"Ye Z, Guntaka RV, Mahato RI",<<title>>\nSequence-specific triple helix form...,<<title>>\nSequence-specific triple helix form...,phosphorothioate\ntriplex forming oligonucleot...,,,phosphorothioate\ntriplex forming oligonucleot...,,rats\nbovine\nrat\n,
6,PMC3078494,Oligo/polynucleotide-based gene modification: ...,2011,Oligonucleotides; 2011 Mar-Apr; 21(2) 55-75. d...,"Sargent RG, Kim S, Gruenert DC",<<title>>\nOligo/polynucleotide-based gene mod...,<<title>>\nOligo/polynucleotide-based gene mod...,Oligonucleotide\npolynucleotide\ngenetic disor...,,genetic disorders\n,Oligonucleotide\npolynucleotide\nOligo/polynuc...,,patient\nmammalian\n,
7,PMC307242,Suppression of c-myc oncogene expression by a ...,1995,Nucleic Acids Res.; 1995 Sep 11; 23(17) 3594-9,"Thomas TJ, Faaland CA, Gallo MA, Thomas T",<<title>>\nSuppression of c-myc oncogene expre...,<<title>>\nSuppression of c-myc oncogene expre...,c-myc\npolyamine\ntriplex forming oligonucleot...,c-myc\nc-myc\nc-myc\nc-myc\nc-myc\n,MCF-7 breast cancer\nbreast cancer\nbreast can...,polyamine\ntriplex forming oligonucleotide\nPo...,,,
8,PMC149218,Selective inhibition of transcription of the E...,2003,Nucleic Acids Res.; 2003 Feb 01; 31(3) 833-43,"Carbone GM, McGuffie EM, Collier A, Catapano CV",<<title>>\nSelective inhibition of transcripti...,<<title>>\nSelective inhibition of transcripti...,Ets2\nprostate cancer\ntriplex-forming oligonu...,Ets2\nEts2\nEts2\nEts2\nEts2\nSp3\nSp3\nSp3\nE...,prostate cancer\ncancer\nprostate cancer\n,triplex-forming oligonucleotide\ntriplex-formi...,,,
9,PM

In [30]:
# 統計

input_data = PMCID_NE_Gene

vectorizer_count = CountVectorizer(lowercase=False)
Feature_TF = vectorizer_count.fit_transform(input_data)
Feature_names = vectorizer_count.get_feature_names()

Data_count = pd.DataFrame(Feature_TF.toarray(),columns=Feature_names,index=Success_ID)

vectorizer_Tfidf = TfidfVectorizer(lowercase=False)
Feature_TFIDF = vectorizer_Tfidf.fit_transform(input_data)
Feature_names = vectorizer_Tfidf.get_feature_names()

Data_paper_tfidf = pd.DataFrame(Feature_TFIDF.toarray(),columns=Feature_names,index=Success_ID)

In [31]:
Data_count

,CCR5,CD1,Ets2,HPRT,Hairless,KRT10,KRT14,RPA,Sp3,TNF,...,chemokine,dystrophin,factor,globin,hairless,keratin,myc,necrosis,progesterone,receptor
PMC3532560,2,2,0,1,1,1,1,1,0,0,...,1,2,0,3,5,8,0,0,0,1
PMC3188747,0,0,0,0,0,0,0,0,0,8,...,0,0,1,0,0,0,0,1,0,0
PMC373298,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PMC309654,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
PMC407813,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
PMC2536773,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PMC3078494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PMC307242,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,0,0,0
PMC149218,0,0,9,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
PMC2841926,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Key word

position = 'PMC3532560'

def search_keyword(position,Data_count):
    series = Data_count.loc[position][ Data_count.loc[position] > np.mean(Data_count.loc[position] [Data_count.loc[position] > 0 ]) ] # 
    if series.empty == False :
        list1, list2 = (list(t) for t in zip(*sorted(zip(series.values, series.index))))
        list1 = list1[::-1]
        list2 = list2[::-1]
        for IPT in range(len(list2)) :
            print('--'+list2[IPT]+'--'+str(list1[IPT]))
        print('\n')

series_raw = Data_count.loc[position][ Data_count.loc[position] > 0 ]
if series_raw.empty == False :
        print(list(series_raw.index))
print('\n')
        
search_keyword(position,Data_count)
search_keyword(position,Data_paper_tfidf)

['CCR5', 'CD1', 'HPRT', 'Hairless', 'KRT10', 'KRT14', 'RPA', 'Troy', 'XPA', 'beta', 'chemokine', 'dystrophin', 'globin', 'hairless', 'keratin', 'receptor']


--keratin--8
--hairless--5
--globin--3
--beta--3


--keratin--0.7078740920295378
--hairless--0.4424213075184611
--globin--0.2654527845110767
--beta--0.2654527845110767


